# ArcGIS API for Python: AIS Route Extraction Prototype

## A collaboration between the U.S. Department of Transportation and Esri

###### Alberto Nieto (Esri), Andrew Barrows (USDOT), Dominic Menegus (USDOT)

This Jupyter Notebook contains documentation, processing, and data visualization of a process in which the Automated Identification System (AIS) vessel data for specified terminal-to-terminal connections is used to digitize a route polyline dataset for an authoritative GIS. 

This process can be leveraged to develop an Information Product that generates data-driven ferry routes for all terminals in the United States, and could be extended to other environments and use cases.

Contact Info:
anieto@esri.com

# Ask Questions

### How can we determine "average" paths for ferry routes in the United States? 

# Proposed Methodology

#### Use AIS positional data to determine ship positions along routes, perform qualitative checks, and digitize polyline geometry

<img src="../../docs/dbclustering.PNG"></img>

#### Script Set-up (Imports, variables, etc.)

In [1]:
import arcgis
import plotly
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib inline  

gis = arcgis.gis.GIS()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



# Part 1: Retrieve and Explore AIS Data

#### Code to Retrieve AIS Data

In [2]:
ais_sample = gis.content.search("AIS_StatenIsland_Sample", item_type="feature layer")[0]
ais_sample

<Item title:"AIS_StatenIsland_Sample" type:Feature Layer Collection owner:albe9057@esri.com_esrifederal>

#### Map of AIS Data

In [3]:
nyc_map = gis.map("New York City")
nyc_map.add_layer(ais_sample)
nyc_map.basemap = 'streets-night-vector'
nyc_map

# Part 2: Run Lowess

#### Explanation of Loess/Lowess

#### Code to run and plot lowess on AIS Staten Island sample

In [4]:
def calculate_lat_lon_in_spatialdf(spatialdf, shape_field='SHAPE', output_lat='latitude', output_lon='longitude'):

    def calculate_latitude(shape_field):
        return shape_field['y']

    def calculate_longitude(shape_field):
        return shape_field['x']

    # Calculate latitude and longitude fields from the shape attribute
    spatialdf[output_lat] = spatialdf.apply(lambda x: calculate_latitude(x[shape_field]), axis=1)
    spatialdf[output_lon] = spatialdf.apply(lambda x: calculate_longitude(x[shape_field]), axis=1)

    return spatialdf

In [5]:
ais_local_fgdb = r"D:\ANieto_SolutionEngineer\Data\DOT\BTS\Zone18_2014_07\Zone18_2014_07.gdb"
ais_sample_fc = "{0}//AIS_Sampler".format(ais_local_fgdb)
ais_sdf = calculate_lat_lon_in_spatialdf(arcgis.features.SpatialDataFrame.from_featureclass(ais_sample_fc))

In [6]:
plotly.plotly.iplot({
    "data": [plotly.graph_objs.Scatter(x=ais_sdf['longitude'], y=ais_sdf['latitude'])]
})

##### Run lowess using Staten Island AIS sample

In [7]:
lowess = sm.nonparametric.lowess
lowess_ais = lowess(ais_sdf['latitude'], ais_sdf['longitude'])

In [8]:
plotly.plotly.iplot({
    "data": [plotly.graph_objs.Scatter(x=lowess_ais[:,0], y=lowess_ais[:,1])]
})

#### Pre-QC Route Layer on WebMap

In [9]:
preqc_route_layer = gis.content.search("AIS_SIFerryRoute_PreQC")[0]
preqc_route_layer

<Item title:"AIS_SIFerryRoute_PreQC" type:Feature Layer Collection owner:albe9057@esri.com_esrifederal>

In [10]:
preqc_route_map = gis.map("Staten Island")
# preqc_route_map.basemap = 'ocean'
# preqc_route_map.add_layer(preqc_route_layer)
preqc_route_map

In [13]:
preqc_route_map.basemap = 'gray'

In [11]:
preqc_route_map.add_layer(preqc_route_layer)

#### Let's add the AIS data to see why this was the case

In [14]:
preqc_route_map.add_layer(ais_sample)

#### Not great... we need to remove noise. 

# Part 3: Run QC - Density-based Clustering

#### Explanation of Density-based Clustering

#### Code to Retrieve Density-based Clustering Layers (with noise, and without noise) and Show on Maps

DBSCAN With noise

In [ ]:
dbcluster_lyr_full = gis.content.search("OPTICS_AIS_Sampler_50ft_lowsens")[0]
dbcluster_map = gis.map("New York City")
dbcluster_map.add_layer(dbcluster_lyr_full)
dbcluster_map

Without noise

In [ ]:
dbcluster_lyr_full = gis.content.search("OPTICS_AIS_Sampler_50ft_lowsens")[0]
dbcluster_map = gis.map("New York City")
dbcluster_map.add_layer(dbcluster_lyr_full)
dbcluster_map

# Part 4: Post QC - Map

#### Lowess Layer on WebMap

In [ ]:
postqc_route_layer = gis.content.search("AIS_SIFerryRoute_PostQC")[0]
postqc_route_layer

In [10]:
postqc_route_map = gis.map("Staten Island")
# preqc_route_map.basemap = 'ocean'
# preqc_route_map.add_layer(preqc_route_layer)
postqc_route_map

In [13]:
postqc_route_map.basemap = 'gray'

In [11]:
postqc_route_map.add_layer(preqc_route_layer)

#### Getting better.

# Part 5: Run at Scale Across the Country

#### Code to Retrieve List of MMSIs

#### Code to run Process

#### Code to show map of processed Ferry Routes 